# Mapping Data with Folium

![](images/folium.png)

Below, we will go through a brief introduction to the **Folium** library.  This is a nice way to build interactive visuzlizations.  We will be executing these in the jupyter notebooks, however they are easily output as `.html` files ready to be served.  To begin, let's make sure we have folium installed.  

In [2]:
import folium
import pandas as pd

We can make basic maps centered at any geolocation.  For example, below we create a basic map of Portland, Oregon.  

In [ ]:
#at its most basic we only have to provide a geo-location
#give it a latitude and longitude

In [3]:
m = folium.Map(location=[45.5236, -122.6750])

In [4]:
m

In [5]:
m.save('index_map.html')

In [ ]:
#all of these are deployable
#you can put this into a webpage with Flask

We can add arguments that include changing the style of the map and the initial zoom level.

In [ ]:
#we can change style and look of folium map, easily
#start at different levels with zoom start
#we can also add markers with the pop up argument

In [6]:
folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=13
)

We can use the `popup` argument to include information to be displayed at specified marker locations.

In [7]:
tooltip = 'Click me!'
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)



folium.Marker([45.3288, -121.6625], popup='<i>Mt. Hood Meadows</i>').add_to(m)
#adding html element to this marker
folium.Marker([45.3311, -121.7113], popup='<b>Timberline Lodge</b>').add_to(m)
m

We can even include `markdown` syntax and icons.

In [ ]:
# we can add markdown
# below we add additional icons to these markers
# we can also circle points of interest
# popup is  the contents of it if you click on it


In [8]:
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[45.3288, -121.6625],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[45.3311, -121.7113],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[45.3300, -121.6823],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

In [9]:
m

We can manually control radii for markers of interest.  Below, we plot two circles at specific locations.

In [91]:
m = folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=13
)

folium.Circle(
    radius=100,
    location=[45.5244, -122.6699],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[45.5215, -122.6261],
    radius=50,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)


In [92]:
m

### Mapping Bike Data

Now, we will use a dataset from NYC's citibike data.  Our goal is to compare incoming and outgoing traffic at given stations depending on the time of day.  

In [10]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")
marker = folium.CircleMarker(location=[40.738, -73.98])
#creating a circle to add to map
marker.add_to(folium_map)

In [11]:
folium_map

In [13]:
bikes = pd.read_csv('data/201306-citibike-tripdata.csv')

In [23]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577703 entries, 0 to 577702
Data columns (total 15 columns):
tripduration               577703 non-null int64
starttime                  577703 non-null object
stoptime                   577703 non-null object
start station id           577703 non-null int64
start station name         577703 non-null object
start station latitude     577703 non-null float64
start station longitude    577703 non-null float64
end station id             559644 non-null float64
end station name           559644 non-null object
end station latitude       559644 non-null float64
end station longitude      559644 non-null float64
bikeid                     577703 non-null int64
usertype                   577703 non-null object
birth year                 337382 non-null float64
gender                     577703 non-null int64
dtypes: float64(6), int64(4), object(5)
memory usage: 66.1+ MB


In [40]:
bikes['starttime'] = pd.to_datetime(bikes['starttime'])
bikes['stoptime'] = pd.to_datetime(bikes['stoptime'])
bikes['hour'] = bikes['starttime'].map(lambda x: x.hour)
bikes['ehour'] = bikes['stoptime'].map(lambda x: x.hour)

In [65]:
bikes.head()[['end station id', 'start station id', 'hour','ehour', 'usertype']]

,end station id,start station id,hour,ehour,usertype
0,434.0,444,0,0,Subscriber
1,434.0,444,0,0,Subscriber
2,406.0,406,0,0,Customer
3,262.0,475,0,0,Subscriber
4,310.0,2008,0,0,Subscriber


In [19]:
bikes['hour'].sort_values(ascending=False)[:4]

361243    23
219737    23
219725    23
219726    23
Name: hour, dtype: int64

In [20]:
locations = bikes.groupby('start station id').first()

In [26]:
locations.head()

,start station latitude,start station longitude,start station name
start station id,,,
72,40.767272,-73.993929,W 52 St & 11 Ave
79,40.719116,-74.006667,Franklin St & W Broadway
82,40.711174,-74.000165,St James Pl & Pearl St
83,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl
116,40.741776,-74.001497,W 17 St & 8 Ave


In [21]:
locations = locations.loc[:, ["start station latitude", "start station longitude", "start station name"]]

In [22]:
subset = bikes[bikes["hour"]==10]

In [23]:
dept_counts = subset.groupby("start station id").count()

In [24]:
dept_counts = dept_counts.iloc[:, [0]]

In [25]:
dept_counts.columns = ["Departure Counts"]

### Problem

Repeat the above for arrivals, in anticipation of joining the two for our map.

In [41]:
locations2 = bikes.groupby('end station id').first()
locations2 = locations2.loc[:, ["end station latitude", "end station longitude", "end station name"]]
subset = bikes[bikes["ehour"]==10]
arr_counts = subset.groupby("end station id").count()
arr_counts = arr_counts.iloc[:, [0]]
arr_counts.columns = ["Arrival Counts"]


In [ ]:
#geo pandas for long/lat look up

In [43]:
trip_counts = dept_counts.join(locations).join(arr_counts)
trip_counts.head()

,Departure Counts,start station latitude,start station longitude,start station name,Arrival Counts
start station id,,,,,
72,80,40.767272,-73.993929,W 52 St & 11 Ave,65
79,117,40.719116,-74.006667,Franklin St & W Broadway,157
82,25,40.711174,-74.000165,St James Pl & Pearl St,17
83,44,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl,61
116,115,40.741776,-74.001497,W 17 St & 8 Ave,132


In [45]:
for index, row in trip_counts.iterrows():
    
    net_departures = (row["Departure Counts"]-row["Arrival Counts"])
    
    radius = net_departures/7
    
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True).add_to(folium_map)

In [46]:
folium_map

In [48]:
popup_text = """{}<br>
                total departures: {}<br> 
                total arrivals: {}<br>
                net departures: {}"""


popup_text = popup_text.format(row["start station name"],
                               row["Arrival Counts"],
                               row["Departure Counts"],
                               net_departures)

In [49]:
for index, row in trip_counts.iterrows():
    net_departures = (row["Departure Counts"]-row["Arrival Counts"])
    radius = net_departures/7
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True, popup = popup_text).add_to(folium_map)

In [50]:
folium_map

### PROBLEM

Compare this image to that of when people are leaving work.  Doe you see what you expect?  What does this tell you about movement in the city?

In [ ]:
locations2 = bikes.groupby('end station id').first()
locations2 = locations2.loc[:, ["end station latitude", "end station longitude", "end station name"]]
subset = bikes[bikes["hour"]==10]
arr_counts = subset.groupby("end station id").count()
arr_counts = arr_counts.iloc[:, [0]]
arr_counts.columns = ["Arrival Counts"]